# Embedding's for MovieLens dataset

## Описание задачи

Необходимо построить векторное представление пользователей и фильмов используя нейросетевые подходы, чтобы можно было по эмбендингу пользователя искать похожие эмбендинги фильмов и рекомендовать ему их.

Обратить внимание на:

1. Какие данных на обучение и валидацию, обоснование.
2. Выбор и обоснование метрики
3. Разработка архитектуры нейронной сети с пояснением
4. Обучение и валидация
5. Решение должно быть воспроизводимым с подробными комментариями на каждом шаге

## Описание данных

*Полный* датасет MovieLens с сайта grouplens.org: https://files.grouplens.org/datasets/movielens/ml-latest.zip

В датасете содержится информация о фильмах, оценках пользователей и тегах фильмов.


Структура файла данных рейтингов (ratings.csv)
-----------------------------------------

Все рейтинги содержатся в файле `ratings.csv`. Каждая строка этого файла после строки заголовка представляет собой одну оценку одного фильма одним пользователем и имеет следующий формат:

    userId,movieId,rating,timestamp

Строки в этом файле упорядочены сначала по идентификатору пользователя, затем, внутри пользователя, по идентификатору фильма.

Оценки выставляются по 5-звездочной шкале, с шагом в ползвезды (0,5 звезды - 5,0 звезды).

Временные метки представляют собой секунды с полуночи по всемирному координированному времени (UTC) от 1 января 1970 года.


Структура файла данных тегов (tags.csv)
-----------------------------------

Все теги содержатся в файле `tags.csv`. Каждая строка этого файла после строки заголовка представляет собой один тег, примененный к одному фильму одним пользователем, и имеет следующий формат:

    userId,movieId,tag,timestamp

Строки в этом файле упорядочены сначала по userId, затем, внутри пользователя, по movieId.

Теги - это созданные пользователем метаданные о фильмах. Каждый тег обычно представляет собой одно слово или короткую фразу. Смысл, значение и цель конкретного тега определяется каждым пользователем.

Временные метки представляют собой секунды с полуночи по всемирному координированному времени (UTC) от 1 января 1970 года.


Структура файла данных фильмов (movies.csv)
---------------------------------------

Информация о фильмах содержится в файле `movies.csv`. Каждая строка этого файла после строки заголовка представляет один фильм и имеет следующий формат:

    movieId,title,genres

Названия фильмов вводятся вручную или импортируются из <https://www.themoviedb.org/> и включают год выпуска в круглых скобках. В этих названиях могут быть ошибки и несоответствия.

Жанры представляют собой список, разделенный трубкой, и выбираются из следующих:

* боевик
* Приключения
* Анимация
* Детские
* Комедия
* Криминал
* Документальный
* Драма
* Фэнтези
* Фильм-нуар
* Ужасы
* Мюзикл
* Мистерия
* Романтика
* Научная фантастика
* Триллер
* Война
* Вестерн
* (жанры не указаны)


Структура файла данных ссылок (links.csv)
---------------------------------------

Идентификаторы, которые могут быть использованы для ссылок на другие источники данных о фильмах, содержатся в файле `links.csv`. Каждая строка этого файла после строки заголовка представляет один фильм и имеет следующий формат:

    movieId,imdbId,tmdbId

movieId - это идентификатор для фильмов, используемых <https://movielens.org>. Например, фильм "История игрушек" имеет ссылку <https://movielens.org/movies/1>.

imdbId - это идентификатор для фильмов, используемых <http://www.imdb.com>. Например, фильм "История игрушек" имеет ссылку <http://www.imdb.com/title/tt0114709/>.

tmdbId - идентификатор для фильмов, используемых <https://www.themoviedb.org>. Например, фильм "История игрушек" имеет ссылку <https://www.themoviedb.org/movie/862>.

Использование перечисленных выше ресурсов регулируется условиями каждого поставщика.


Геном тегов (genome-scores.csv и genome-tags.csv)
-------------------------------------------------

Этот набор данных включает текущую копию генома тегов.

[genome-paper]: http://files.grouplens.org/papers/tag_genome.pdf

Геном тегов - это структура данных, которая содержит оценки релевантности тегов для фильмов.  Структура представляет собой плотную матрицу: каждый фильм в геноме имеет значение для *каждого* тега в геноме.

Как описано в [этой статье][genome-paper], геном тегов кодирует, насколько сильно фильмы проявляют определенные свойства, представленные тегами (атмосферность, заставляющие задуматься, реалистичность и т.д.). Геном тегов был вычислен с помощью алгоритма машинного обучения на пользовательском контенте, включая теги, рейтинги и текстовые рецензии.

Геном разделен на два файла.  Файл `genome-scores.csv` содержит данные о релевантности тегов фильма в следующем формате:

    movieId,tagId,relevance

Второй файл, `genome-tags.csv`, содержит описания тегов для идентификаторов тегов в файле генома в следующем формате:

    tagId,tag

Значения `tagId` генерируются при экспорте набора данных, поэтому они могут отличаться в разных версиях наборов данных MovieLens.

При ссылке на геномные данные тегов, пожалуйста, указывайте следующую цитату:

> Jesse Vig, Shilad Sen, and John Riedl. 2012. The Tag Genome: Кодирование знаний сообщества для поддержки новых взаимодействий. ACM Trans. Interact. Intell. Syst. 2, 3: 13:1-13:44. <https://doi.org/10.1145/2362394.2362395>

## Анализ и подготовка данных

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [14]:
from pyspark.sql import SparkSession
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark

In [15]:
movies = spark.read.csv('ml-latest/movies.csv', header=True, inferSchema=True)
ratings = spark.read.csv('ml-latest/ratings.csv', header=True, inferSchema=True)

In [16]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [17]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
|     1|   1590|   2.5|1256677236|
|     1|   1591|   1.5|1256677475|
|     1|   2134|   4.5|1256677464|
|     1|   2478|   4.0|1256677239|
|     1|   2840|   3.0|1256677500|
|     1|   2986|   2.5|1256677496|
|     1|   3020|   4.0|1256677260|
|     1|   3424|   4.5|1256677444|
|     1|   3698|   3.5|1256677243|
|     1|   3826|   2.0|1256677210|
|     1|   3893|   3.5|1256677486|
|     2|    170|   3.5|1192913581|
|     2|    849|   3.5|1192913537|
|     2|   1186|   3.5|1192913611|
|     2|   1235|   3.0|1192913585|
+------+-------+------+----------+
only showing top 20 rows



In [18]:
# merge ratings and movies by movieId and sort by userId
df = ratings.join(movies, on='movieId', how='left').sort('userId')

In [19]:
del ratings, movies

In [20]:
df.show()

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|    307|     1|   3.5|1256677221|Three Colors: Blu...|               Drama|
|    481|     1|   3.5|1256677456|   Kalifornia (1993)|      Drama|Thriller|
|   1091|     1|   1.5|1256677471|Weekend at Bernie...|              Comedy|
|   1257|     1|   4.5|1256677460|Better Off Dead.....|      Comedy|Romance|
|   1449|     1|   4.5|1256677264|Waiting for Guffm...|              Comedy|
|   1590|     1|   2.5|1256677236|Event Horizon (1997)|Horror|Sci-Fi|Thr...|
|   1591|     1|   1.5|1256677475|        Spawn (1997)|Action|Adventure|...|
|   2134|     1|   4.5|1256677464|Weird Science (1985)|Comedy|Fantasy|Sc...|
|   2478|     1|   4.0|1256677239|¡Three Amigos! (1...|      Comedy|Western|
|   2840|     1|   3.0|1256677500|     Stigmata (1999)|      Drama|Thriller|

In [21]:
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [22]:
# switch type of column rating from double to float
from pyspark.sql.types import FloatType
df = df.withColumn("rating", df["rating"].cast(FloatType()))

In [23]:
# show statistics for all data
ratings_pre_user = df.groupBy('userId').count().select('count').toPandas()
ratings_pre_movie = df.groupBy('movieId').count().select('count').toPandas()

print('Total No. of Users: {}'.format(ratings_pre_user['count'].sum()))
print('Total No. of Movies: {}'.format(ratings_pre_movie['count'].sum()))
print()

print('Max observed rating: {}'.format(df.agg({"rating": "max"}).collect()[0][0]))
print('Min observed rating: {}'.format(df.agg({"rating": "min"}).collect()[0][0]))
print()

print('Max No. of ratings per user: {}'.format(ratings_pre_user['count'].max()))
print('Min No. of ratings per user: {}'.format(ratings_pre_user['count'].min()))
print('Median No. of ratings per user: {}'.format(ratings_pre_user['count'].median()))
print()

print('Max No. of ratings per movie: {}'.format(ratings_pre_movie['count'].max()))
print('Min No. of ratings per movie: {}'.format(ratings_pre_movie['count'].min()))
print('Median No. of ratings per movie: {}'.format(ratings_pre_movie['count'].median()))

del ratings_pre_user, ratings_pre_movie

Total No. of Users: 27753444
Total No. of Movies: 27753444

Max observed rating: 5.0
Min observed rating: 0.5

Max No. of ratings per user: 23715
Min No. of ratings per user: 1
Median No. of ratings per user: 30.0

Max No. of ratings per movie: 97999
Min No. of ratings per movie: 1
Median No. of ratings per movie: 7.0


In [24]:
# rename columns movieId and userId to movie_id and user_id
df = df.withColumnRenamed('movieId', 'movie_id').withColumnRenamed('userId', 'user_id')

In [38]:
from pyspark.sql.functions import col, rank, dense_rank, count
from pyspark.sql.window import Window


def get_last_n_ratings_by_user(
        df, n, min_ratings_per_user=1, user_colname="user_id", timestamp_colname="timestamp"
):
    # Count the number of ratings per user
    user_counts = df.groupby(user_colname).agg(count("*").alias("user_count"))

    # Filter out users with less than min_ratings_per_user
    valid_users = user_counts.filter(col("user_count") >= min_ratings_per_user)

    # Join the filtered user counts with the original DataFrame
    joined_df = df.join(valid_users, on=user_colname)

    # Rank the ratings within each user group based on the timestamp
    ranked_df = joined_df.withColumn(
        "rank", dense_rank().over(Window.partitionBy(user_colname).orderBy(col(timestamp_colname).desc()))
    )

    # Filter out ratings that are not in the last n for each user
    last_n_ratings_df = ranked_df.filter(col("rank") <= n)

    # Sort the resulting DataFrame by user_id and timestamp
    sorted_df = last_n_ratings_df.sort(user_colname, timestamp_colname)

    return sorted_df



In [39]:
def mark_last_n_ratings_as_validation_set(
        df, n, min_ratings=1, user_colname="user_id", timestamp_colname="timestamp"
):
    w = Window.partitionBy(user_colname).orderBy(col(timestamp_colname).desc())
    ranked_df = df.withColumn("rank", rank().over(w))
    filtered_df = ranked_df.filter(col("rank") <= n)
    filtered_users_df = filtered_df.groupBy(user_colname).filter(
        lambda x: x.count() >= min_ratings
    )
    valid_indices = [
        row["index"]
        for row in filtered_users_df.select("index").collect()
    ]
    df = df.withColumn("is_valid", col("index").isin(valid_indices))
    return df


In [40]:
marked_df = mark_last_n_ratings_as_validation_set(df, n=5, min_ratings=30)

AttributeError: 'GroupedData' object has no attribute 'filter'

In [ ]:
train_df = df.filter(df.is_valid == False).drop('is_valid')
test_df = df.filter(df.is_valid == True).drop('is_valid')

In [25]:
# show ratio of train and test data
print('Train data ratio: {}'.format(train_df.count() / df.count()))